In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plotwidth=40

In [ ]:
import pandas as pd
import math
from natsort import natsort_keygen
import glob
import yaml
import numpy as np
import datetime
import pandas as pd
import math
from natsort import natsort_keygen
import glob
import yaml
import numpy as np
import datetime
import statsmodels.api as sm
from tqdm.notebook import tqdm, trange
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression
from scipy.optimize import nnls
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
import json
import psycopg2
import netrc
import re
import os
from multiprocess import Pool

lowess = sm.nonparametric.lowess

# Globals

In [ ]:
datadir = '.'
plotdir = 'plots'
vpipe_working = 'working' # V-pipe's working directory


tally_data = os.path.join(datadir, 'tallymut_line.tsv')
plant_name_tsv = pd.read_csv('ww_plants.tsv')

#for later
# cooc_data = 'data/ww-cooc.csv'

start_date = '2020-12-08'
todaydate = datetime.date.today().strftime("%Y-%m-%d")
cities_list=['Altenrhein (SG)', 'Chur (GR)', 'Genève (GE)', 'Kanton Zürich', 'Laupen (BE)',
       'Lausanne (VD)', 'Lugano (TI)', 'Zürich (ZH)']
#cities_list=['Kanton Zürich','Kanton Zürich/Promega']
variants_pangolin={'UK':'B.1.1.7','ZA':'B.1.351','BR':'P.1','C36':'C.36.3','IN1':'B.1.617.1','IN2':'B.1.617.2','IN3':'B.1.617.3'}
variants = ['P.1', 'C.36.3', 'B.1.617.1', 'B.1.617.2', 'B.1.617.3', 'B.1.1.7', 'B.1.351']
exclusive_list=['B.1.351','P.1'] # list of variants where we should apply filtering
exclude_from=['B.1.1.7','B.1.351','P.1' ] #,'IN2'] #,'C36','IN1','IN2','IN3'] # filter against these variants
max_pool = len(cities_list)

# mutations type to be considered
mut_types = ['mut', 'extra']
# drop all shared and subset mutations
to_drop = ['subset', 'shared']


# Outputs
plots_dir='deconv_plots'
if not os.path.isdir(plots_dir):
    try:
        os.mkdir(plots_dir, mode=0o775)
    except FileExistsError:
        pass

# Load Data

In [ ]:
# load tsv into dataframe
df_tally = pd.read_csv(tally_data, sep='\t')
df_tally = df_tally.rename(columns=variants_pangolin)
df_tally = df_tally.drop('EU', axis=1)

In [ ]:
df_data = df_tally.copy()
df_data.dropna(subset = ["frac", "date"], inplace=True)
df_data['mutations'] = df_data['pos'].astype(str) + df_data['base']
df_data = df_data[~(df_data['base'] == '-') & (df_data['date'] >= start_date) ]
df_data = df_data[df_data.columns.difference(['pos', 'gene', 'base'], sort=False)]

# drop other mutation type from df_data
for v in variants:
    df_data = df_data[~df_data[v].isin(to_drop)]

df_data = df_data.reset_index(drop=True)

mutations =  sorted(list(df_data['mutations'].unique()), key=natsort_keygen())
# df_data[df_data['plantname'] == 'Altenrhein (SG)'].sort_values(by='date')

In [ ]:
df_data2 = df_data[df_data.columns.difference(['batch'], sort=False)]
df_data2 = df_data2.replace(np.nan, 0)
df_data2 = df_data2.replace(['extra', 'mut'], 1)
df_data2 = df_data2[df_data2.columns.difference(['plantcode', 'cov', 'var'], sort=False)]
df_data2 = df_data2.sort_values(by=['date', 'sample'])

dates = sorted(set(df_data2['date']), key=natsort_keygen())
# df_data2[(df_data2['plantname'] == 'Lugano (TI)') & (df_data2['date'] == '2021-07-20')]
df_data2.insert(4, 'undetermined', 0)
# sorted(df_data2['date'].unique())

In [ ]:
#Complement of matrix A (to add undetermined case)
data = {'sample':df_data2['sample'], 'date':df_data2['date'], 'plantname':df_data2['plantname'],
       'frac':1-df_data2['frac'], 'P.1':1-df_data2['P.1'], 'C.36.3':1-df_data2['C.36.3'], 
        'B.1.617.1':1-df_data2['B.1.617.1'], 'B.1.617.2':1-df_data2['B.1.617.2'], 
        'B.1.617.3':1-df_data2['B.1.617.3'],'B.1.1.7':1-df_data2['B.1.1.7'], 
        'B.1.351':1-df_data2['B.1.351'], 'mutations': '-' + df_data2['mutations'].astype(str)}

df_data3 = pd.DataFrame(data)
df_data3.insert(4, 'undetermined', 1)
df_data3

In [ ]:
df_data4 = pd.concat([df_data2, df_data3], sort=False)
df_data4[df_data4['undetermined'] == 0]
mutations =  sorted(list(df_data4['mutations'].unique()), key=natsort_keygen())
df_data4[df_data4['B.1.617.2'] == 1]
variants = ['undetermined', 'P.1', 'C.36.3', 'B.1.617.1', 'B.1.617.2', 'B.1.617.3', 'B.1.1.7', 'B.1.351']
variants = sorted(variants, key=natsort_keygen())

# sorted(df_data4[df_data4['plantname'] == cities_list[2]]['date'].unique())
df_data4

# Ridge Regression

In [ ]:
def ineq_cons(x):
    """constrain all elements of x to be >= 0"""
    return x

def eq_cons(x):
    """constrain the sum of all rows to be equal to 1"""
    return np.sum(x) - 1

def fn(x, A, b):
    return 0.5*np.linalg.norm(A.dot(x)-b)**2

def cal_nnls_ridge2(X,y, lam, l=8):
    p = X.shape[1]
    Xext = np.vstack((X, lam * np.identity(X.shape[1])))

    yext = np.hstack((y, np.zeros(p)))
    coefs, _ = nnls(Xext, yext)

    cons = [{'type': 'ineq', 'fun': ineq_cons},
               {'type': 'eq', 'fun': eq_cons}]

    #Call minimisation subject to these values
    minout = minimize(fn, coefs, args=(Xext, yext), method='SLSQP',bounds=[(0., None) for i in range(l)]
                      ,constraints=cons)

    x = minout.x

    return x    

In [ ]:
def regress_city(city): # globals: df_data2, df_data4, variants
    current_ridge = []
#     df_city1 = df_data2[(df_data2['plantname'] == city)]
    df_city1 = df_data4[(df_data4['plantname'] == city)]
    dates = sorted(set(df_city1['date']), key=natsort_keygen())

    for n in trange(0,100, desc=city):
        for d in dates:
            muts = np.random.choice(mutations, len(mutations), replace=True)

            df_city_date = df_city1[df_city1['date'] == d]
            df_sampled = df_city_date[df_city_date['mutations'].map(lambda x: x in muts)]

            if len(df_sampled) > 0:
                df_ridge = df_sampled[df_sampled.columns.difference(['sample', 'date', 'plantname', 'mutations'], sort=False)]

                char_vars = df_ridge[df_ridge.columns.difference(['sample', 'date','plantname',
                                                                          'frac', 'mutations', 'date'])].columns.values

                for value, variant in zip(cal_nnls_ridge2(np.array(df_ridge[variants]), np.array(df_ridge['frac']), 0.5,
                                                              len(char_vars)), char_vars):
                        current_ridge.append([variant, value, city, d, n])
    return current_ridge

In [ ]:
ridge_res=[]
# city_ex = [cities_list[0]]#for testing

#with Pool(max_pool) as p:
#    global df_data2, df_data4, variants
#    pool_res = list(
#        tqdm(
#            p.imap(regress_city,
#                   cities_list),
#            total=len(cities_list)
#        )
#    )
#for res in pool_res:
#    ridge_res += res

for city in tqdm(cities_list, desc='Cities', position=0):
    ridge_res += regress_city(city)
            

In [ ]:
# load regression coefs to dataframe
df_ridge_res = pd.DataFrame(ridge_res)
df_ridge_res = df_ridge_res.rename(columns={0: 'variant', 1: 'weight', 2: 'location', 3: 'date', 4: 'iter'})
df_ridge_res['date'] = pd.to_datetime(df_ridge_res['date'])

df_final = df_ridge_res.sort_values(by=['date','variant']).reset_index(drop=True)
df_final.head()

# Smoothing

In [ ]:
#Smoothing
agg={}
agg2={}
agg3={}
df_smooth1={}
df_smooth2={}
d_test={}
#city_ex =[cities_list[0], cities_list[3]]
for city in tqdm(cities_list, desc='Cities', position=0):
    agg[city]={}
    agg2[city]={}
    agg3[city]={}
    df_smooth1[city]={}
    df_smooth2[city]={}
    d_test[city]={}

    for var in tqdm(variants, desc=city, position=1, leave=False):

        df_smooth1[city][var] = df_final[(df_final['location'] == city) &
                                         (df_final['variant'] == var)]

        df_smooth1[city][var] = df_smooth1[city][var].groupby(['variant', 'location', 'date'])['weight'].apply(list)

        df_smooth2[city][var] = df_smooth1[city][var].apply(pd.Series)

        agg2[city][var] = df_smooth2[city][var].reset_index()
        agg3[city][var] = (df_smooth2[city][var].apply(lambda x: lowess(x, np.arange(x.shape[0]).astype('float64'),
                                                            xvals = np.arange(x.shape[0]).astype('float64'),
                                                            frac= np.clip(20./df_smooth2[city][var].shape[0], 0, 2./3), it=0), 0))
#                           ) if city != 'Kanton Zürich' else (
#                             # For KLZH data
#                            df_smooth2[city][var].apply(lambda x: x.rolling(window=2).mean(), 0)
#                           )
        

# Prevalence Plots

In [ ]:

# show all variants separately
sns.set_palette("dark")
variants = sorted(variants, key=natsort_keygen())
for city in tqdm(cities_list, desc='Cities', position=0):

#     print(city)
#     fig = plt.figure()
#     fig, ax = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)
#     ax = [ax]

    for var in tqdm(variants, desc=city, position=1, leave=False):

        fig = plt.figure()
        fig, ax = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)
        ax = [ax]

        # TODO be more clever with amplicons (for now we're just ignoring them)
        xvals = sorted(list(set(agg2[city][var]['date'])))


        sns.lineplot(x=xvals, y=np.clip(agg3[city][var].apply(np.nanmean, 1), 0., 1.),
                     ax=ax[0], markers=True, label=var, linewidth=1)

        ax[0].fill_between(xvals,
                           np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 5), 1).interpolate(), 0, 1),
                           np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 95), 1).interpolate(), 0, 1),
                           alpha=.4)

        ax[0].set_xlim((np.datetime64(start_date), np.datetime64(todaydate)))
        ax[0].set_ylabel(f"coeffs")
#         ax[0].legend(loc="upper left")
        ax[0].set_title(f"{city}: smoothed regression curve")
#        plt.savefig(os.path.join(plots_dir, f'{city}_{var}.png')




In [ ]:
# show all variants at the same time
sns.set_palette("dark")
variants = sorted(variants, key=natsort_keygen())
for city in tqdm(cities_list, desc='Cities', position=0):

    fig = plt.figure()
    fig, ax = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)
    ax = [ax]

    for var in tqdm(variants, desc=city, position=1, leave=False):

        # TODO be more clever with amplicons (for now we're just ignoring them)
        xvals = sorted(list(set(agg2[city][var]['date'])))


        sns.lineplot(x=xvals, y=np.clip(agg3[city][var].apply(np.nanmean, 1), 0., 1.),
                     ax=ax[0], markers=True, label=var, linewidth=1)

#         ax[0].fill_between(xvals,
#                            np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 5), 1).interpolate(), 0, 1),
#                            np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 95), 1).interpolate(), 0, 1),
#                            alpha=.4)

        ax[0].set_xlim((np.datetime64(start_date), np.datetime64(todaydate)))
        ax[0].set_ylabel(f"coeffs")
#         ax[0].legend(loc="upper left")
        ax[0].set_title(f"{city}: smoothed regression curve")
    plt.savefig(os.path.join(plots_dir, f"{city.replace('/','-')}.png"))



In [ ]:
#df_data2[(df_data2['plantname'] == 'Lugano (TI)') & (df_data2['date'] == '2021-05-19')]
df_data2[(df_data2['plantname'] == 'Kanton Zürich') & (df_data2['date'] == '2021-08-22')]

# Make data for covSPECTRUM

Actually, we are cheating, we are not really preparing the data for cov-spectrum, we are re-using the data that was created with the classical.

You neet to have run [`ww_smoothing_cov.ipynb` all the way up to section New regression](ww_smoothing_cov.ipynb#New-regression) in order to generate the remaining upload data (heatmaps, etc.)

**TODO** clean the heatmap generation code, and add it here.

In [ ]:
#from IPython.display import JSON

old_json='ww_update_data.json'
print("reusing %s last modified: %s" % (old_json, time.ctime(os.path.getmtime(old_json))))
with open(old_json, 'r') as file:
     old_update_data = json.load(file)
#JSON(old_update_data) # only in Jupyter Lab

## Replace the curve with the new regressions above

In [ ]:
update_data={ }
tdf={city:{}  for city in cities_list}
tdf_mat={city:{}  for city in cities_list}

# HACK do not upload 'undetermined' for now
variants = [v for v in variants if v != 'undetermined']

# this next line clips the plots from a given date
only_start_from={'Kanton Zürich':'2021-08-15'} # start_date

for var in tqdm(variants, desc='Variants', position=0):
    update_data[var] = { }
    for city in tqdm(cities_list, desc=var, position=1, leave=False):
        tdf[city][var] = agg3[city][var].apply(lambda x: {"proportion":np.clip(np.mean(x), 0., 1.),
                                                          "proportionLower":np.clip(np.percentile(x, 5), 0., 1.),
                                                          "proportionUpper":np.clip(np.percentile(x, 95), 0., 1.)},
                                               axis=1, result_type ='expand')
        tdf[city][var] = tdf[city][var].reset_index()
        tdf[city][var]["date"] = tdf[city][var]["date"].astype("str")

        #tdf_mat[city][var] = m_df[city][var].T.melt(ignore_index=False, var_name="nucMutation", value_name="proportion").reset_index()
        #tdf_mat[city][var]["date"] = tdf_mat[city][var]["date"].astype("str")
        # # drawing box decorations:
        # # ┌22917G
        # # ├22995A
        # # └Amplicon 76
        #if var in amplicons:
        #    # mutations
        #    for amp,muts in amplicons[var].items():
        #        box = '┌'
        #        for m in muts:
        #            tdf_mat[city][var].loc[tdf_mat[city][var]["nucMutation"]==m,"nucMutation"]=f"{box}{m}"
        #            box = '├'
        #    for a in amp_col[var]:
        #            tdf_mat[city][var].loc[tdf_mat[city][var]["nucMutation"]==a,"nucMutation"]=f"└{a}"

        update_data[var][city] = {
            #"updateDate": todaydate,
            "timeseriesSummary": [dict(tdf[city][var].iloc[i,]) for i in range(tdf[city][var].shape[0]) if (city not in only_start_from) or (tdf[city][var].loc[i,'date'] >= only_start_from[city])],
            "mutationOccurrences": [ x for x in old_update_data[var][city]["mutationOccurrences"] if (city not in only_start_from) or (x['date'] >= only_start_from[city])], # old_update_data[var][city]["mutationOccurrences"], #[dict(tdf_mat[city][var].iloc[i,]) for i in range(tdf_mat[city][var].shape[0])],
        }

import json
with open('ww_update_data_regression.json', 'w') as file:
     file.write(json.dumps(update_data))

In [ ]:
# check the generated curve data for one example
tdf['Zürich (ZH)']['B.1.1.7']

In [ ]:
# check the uploaded curve data for one example
# NOTE this is clipped using only_start_from
#update_data['B.1.1.7']['Zürich (ZH)']["timeseriesSummary"]
update_data['B.1.1.7']['Kanton Zürich']["timeseriesSummary"]

In [ ]:
# check the (copy-pasted) heatmap data for one example
# NOTE this is clipped using only_start_from
#update_data['B.1.1.7']['Zürich (ZH)']["mutationOccurrences"]
update_data['B.1.1.7']['Kanton Zürich']["mutationOccurrences"]

## Upload to Cov-Spectrum

In [ ]:
dbhost='id-hdb-psgr-cp61.ethz.ch'

In [ ]:
# load from netrc
dbuser,dbpass=netrc.netrc().authenticators(dbhost)[0::2]

# alternative: input box
#dbuser = input(f"Enter username for database {dbhost}:\n")
#dbpass = input(f"Enter password for user {dbuser}:\n")

# alternative: enviro
#dbuser = os.environ['DB_USERNAME'],
#dbpass = os.environ['DB_PASSWORD'],

dbuser

In [ ]:
dbconn = psycopg2.connect(
    host=dbhost,
    database='sars_cov_2',
    user=dbuser,
    password=dbpass,
    port='5432'
)
dbconn

In [ ]:
cur = dbconn.cursor()
cur

In [ ]:
for pango in variants:
    for city in cities_list:
        cur.execute("""
        DO $$
        BEGIN
         IF EXISTS (SELECT ww.data FROM public.spectrum_waste_water_result AS ww WHERE ww.variant_name=%(var)s AND ww.location=%(city)s) THEN
          UPDATE public.spectrum_waste_water_result AS ww SET data=%(data)s WHERE ww.variant_name=%(var)s AND ww.location=%(city)s;
         ELSE
          INSERT INTO public.spectrum_waste_water_result (variant_name, location, data)
          VALUES(%(var)s, %(city)s, %(data)s);
         END IF;
        END
        $$
        """, {'data': json.dumps(update_data[pango][city]).replace('NaN','null'), 'var': pango, 'city': city})

In [ ]:
## Abort DB update !
dbconn.rollback()

In [ ]:
## Save to DB !
dbconn.commit()

In [ ]:
cur.close()
dbconn.close()